In [1]:
import numpy as np
import cv2

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

from srgan import SRGAN
from generator import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

In [3]:
input_dir = 'data/train/low/'
tgt_dir = 'data/train/high/'

test_file = 'UBMk30rjy0o_17675_1.jpg'
test_low_path =  'data/val/low/' + test_file
test_high_path = 'data/val/high/' + test_file
test_out_path = 'data/out/' + test_file
test_test_path = 'data/test/' + test_file

vgg_path = 'vgg16_notop.hdf5'

input_shape = (135, 240, 3)
tgt_shape = (540, 960, 3)

batch_size = 4

In [4]:
srgan = SRGAN(vgg_path, batch_size, input_shape, tgt_shape)
gen = ImageDataGenerator(input_dir, tgt_dir, batch_size)

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
test_img_low = ((cv2.imread(test_low_path) - 127.5) / 127.5)[np.newaxis, :, :, :]
test_img_high = ((cv2.imread(test_high_path)- 127.5) / 127.5)[np.newaxis, :, :, :]

test_img_sr = srgan.generator.predict(test_img_low)
test_img_sr_int = ((test_img_sr * 127.5) + 127.5).astype('u1')[0]
cv2.imwrite(test_test_path, test_img_sr_int)

True

In [6]:
for e in range(5):
    print('epoch', e)
    X_low, X_high = gen.flow_from_directory().__next__()
    srgan.train(X_low, X_high)

epoch 0


ResourceExhaustedError:  OOM when allocating tensor with shape[4,540,960,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node SquaredDifference_27-0-0-TransposeNCHWToNHWC-LayoutOptimizer}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_keras_scratch_graph_19056]

Function call stack:
keras_scratch_graph


In [8]:
for e in range(300):
    print('epoch', e)
    data = gen.flow_from_directory().__next__()
    X_low = data[0]
    X_high = data[1]
    srgan.train(X_low, X_high)

epoch 0
d_loss 0.6181113719940186
epoch 1
d_loss 0.623425304889679
epoch 2
d_loss 0.6893672347068787
epoch 3
d_loss 0.6618378162384033
epoch 4
d_loss 0.674908459186554
epoch 5
d_loss 0.6163884401321411
epoch 6
d_loss 0.6246289014816284
epoch 7
d_loss 0.7069726586341858
epoch 8
d_loss 0.6114041805267334
epoch 9
d_loss 0.6736556887626648
epoch 10
d_loss 0.6871210336685181
epoch 11
d_loss 0.6945561170578003
epoch 12
d_loss 0.6211631894111633
epoch 13
d_loss 0.662213921546936
epoch 14
d_loss 0.6196154952049255
epoch 15
d_loss 0.6379116773605347
epoch 16
d_loss 0.7054023146629333
epoch 17
d_loss 0.674554169178009
epoch 18
d_loss 0.7110623717308044
epoch 19
d_loss 0.6532878279685974
epoch 20
d_loss 0.6886740922927856
epoch 21
d_loss 0.7100696563720703
epoch 22
d_loss 0.6412782669067383
epoch 23
d_loss 0.6445339918136597
epoch 24
d_loss 0.6390447616577148
epoch 25
d_loss 0.6458838582038879
epoch 26
d_loss 0.6263339519500732
epoch 27
d_loss 0.7179872393608093
epoch 28
d_loss 0.5916368365287781

d_loss 0.4286300837993622
epoch 232
d_loss 0.4420667886734009
epoch 233
d_loss 0.4423062801361084
epoch 234
d_loss 0.4477206766605377
epoch 235
d_loss 0.44497203826904297
epoch 236
d_loss 0.45808202028274536
epoch 237
d_loss 0.4229184687137604
epoch 238
d_loss 0.43657904863357544
epoch 239
d_loss 0.621127724647522
epoch 240
d_loss 0.6552977561950684
epoch 241
d_loss 0.45550423860549927
epoch 242
d_loss 0.4302237033843994
epoch 243
d_loss 0.43615782260894775
epoch 244
d_loss 0.4444953203201294
epoch 245
d_loss 0.4699496924877167
epoch 246
d_loss 0.6510842442512512
epoch 247
d_loss 0.518570065498352
epoch 248
d_loss 0.44627153873443604
epoch 249
d_loss 0.6632779836654663
epoch 250
d_loss 0.41510042548179626
epoch 251
d_loss 0.5065633654594421
epoch 252
d_loss 0.46488437056541443
epoch 253
d_loss 0.3992735743522644
epoch 254
d_loss 0.5072267055511475
epoch 255
d_loss 0.8381215929985046
epoch 256
d_loss 0.7733257412910461
epoch 257
d_loss 0.7314306497573853
epoch 258
d_loss 0.4902120232582

In [9]:
for e in range(300):
    print('epoch', e)
    data = gen.flow_from_directory().__next__()
    X_low = data[0]
    X_high = data[1]
    srgan.train(X_low, X_high)

epoch 0
d_loss 0.39597558975219727
epoch 1
d_loss 0.5304909944534302
epoch 2
d_loss 0.4395718276500702
epoch 3
d_loss 0.5215224027633667
epoch 4
d_loss 0.5594314336776733
epoch 5
d_loss 0.6165004968643188
epoch 6
d_loss 0.4410811960697174
epoch 7
d_loss 0.4018663763999939
epoch 8
d_loss 0.5353365540504456
epoch 9
d_loss 0.799946129322052
epoch 10
d_loss 0.3756040930747986
epoch 11
d_loss 0.6670982837677002
epoch 12
d_loss 0.6799257397651672
epoch 13
d_loss 0.42741066217422485
epoch 14
d_loss 0.41102588176727295
epoch 15
d_loss 0.37047576904296875
epoch 16
d_loss 0.4418758451938629
epoch 17
d_loss 0.4229797124862671
epoch 18
d_loss 0.455262690782547
epoch 19
d_loss 0.4033775329589844
epoch 20
d_loss 0.39344048500061035
epoch 21
d_loss 0.3929494619369507
epoch 22
d_loss 0.591873049736023
epoch 23
d_loss 0.8030902147293091
epoch 24
d_loss 0.4402567148208618
epoch 25
d_loss 0.4327717423439026
epoch 26
d_loss 0.41391366720199585
epoch 27
d_loss 0.3904991149902344
epoch 28
d_loss 0.503925800

d_loss 0.7268756628036499
epoch 231
d_loss 0.7323498725891113
epoch 232
d_loss 0.7221062183380127
epoch 233
d_loss 0.700251579284668
epoch 234
d_loss 0.6887726783752441
epoch 235
d_loss 0.722901463508606
epoch 236
d_loss 0.4948102831840515
epoch 237
d_loss 0.6520249843597412
epoch 238
d_loss 0.7736943960189819
epoch 239
d_loss 0.9658308029174805
epoch 240
d_loss 0.7318357825279236
epoch 241
d_loss 0.7364596724510193
epoch 242
d_loss 0.6925240159034729
epoch 243
d_loss 0.7247145175933838
epoch 244
d_loss 0.7453925609588623
epoch 245
d_loss 0.6834129691123962
epoch 246
d_loss 0.6908501982688904
epoch 247
d_loss 0.7141355276107788
epoch 248
d_loss 0.7785138487815857
epoch 249
d_loss 0.7431244254112244
epoch 250
d_loss 0.7219855189323425
epoch 251
d_loss 0.7095108032226562
epoch 252
d_loss 0.6706894636154175
epoch 253
d_loss 0.6640652418136597
epoch 254
d_loss 0.8229772448539734
epoch 255
d_loss 0.8120665550231934
epoch 256
d_loss 0.6932907104492188
epoch 257
d_loss 0.5247886180877686
epoc

In [10]:
for e in range(300):
    print('epoch', e)
    data = gen.flow_from_directory().__next__()
    X_low = data[0]
    X_high = data[1]
    srgan.train(X_low, X_high)

epoch 0
d_loss 0.7310494184494019
epoch 1
d_loss 0.9272258281707764
epoch 2
d_loss 0.698296070098877
epoch 3
d_loss 0.9986585974693298
epoch 4
d_loss 0.924297034740448
epoch 5
d_loss 0.7160964012145996
epoch 6
d_loss 0.7189580202102661
epoch 7
d_loss 0.728611171245575
epoch 8
d_loss 0.7693494558334351
epoch 9
d_loss 0.7241849899291992
epoch 10
d_loss 0.9891711473464966
epoch 11
d_loss 0.847419023513794
epoch 12
d_loss 0.7347081899642944
epoch 13
d_loss 0.7130287885665894
epoch 14
d_loss 0.7277429103851318
epoch 15
d_loss 0.7764166593551636
epoch 16
d_loss 0.684158205986023
epoch 17
d_loss 0.7190741300582886
epoch 18
d_loss 0.9144037961959839
epoch 19
d_loss 0.7054049968719482
epoch 20
d_loss 0.9139178395271301
epoch 21
d_loss 0.7398450374603271
epoch 22
d_loss 0.7289597392082214
epoch 23
d_loss 0.6949272751808167
epoch 24
d_loss 0.7318724393844604
epoch 25
d_loss 0.7265743017196655
epoch 26
d_loss 0.8762224912643433
epoch 27
d_loss 0.7194879651069641
epoch 28
d_loss 0.8373282551765442


d_loss 0.7112106084823608
epoch 233
d_loss 0.8053199648857117
epoch 234
d_loss 0.7420376539230347
epoch 235
d_loss 0.7264261245727539
epoch 236
d_loss 0.560847282409668
epoch 237
d_loss 0.6523900628089905
epoch 238
d_loss 0.739308774471283
epoch 239
d_loss 0.6958868503570557
epoch 240
d_loss 0.7069966793060303
epoch 241
d_loss 0.7104225158691406
epoch 242
d_loss 0.7108728885650635
epoch 243
d_loss 0.7106461524963379
epoch 244
d_loss 0.6207306385040283
epoch 245
d_loss 0.721518337726593
epoch 246
d_loss 0.7341879606246948
epoch 247
d_loss 0.7112916707992554
epoch 248
d_loss 0.7482991218566895
epoch 249
d_loss 0.5417584180831909
epoch 250
d_loss 0.6167834997177124
epoch 251
d_loss 0.719059944152832
epoch 252
d_loss 0.7058786153793335
epoch 253
d_loss 0.7996840476989746
epoch 254
d_loss 0.7102718949317932
epoch 255
d_loss 0.7246204614639282
epoch 256
d_loss 0.7177793979644775
epoch 257
d_loss 0.6753963232040405
epoch 258
d_loss 0.7149922847747803
epoch 259
d_loss 0.7340238094329834
epoch 

In [12]:
test_img_low = ((cv2.imread(test_low_path) - 127.5) / 127.5)[np.newaxis, :, :, :]
test_img_high = ((cv2.imread(test_high_path)- 127.5) / 127.5)[np.newaxis, :, :, :]

test_img_sr = srgan.generator.predict(test_img_low)
test_img_sr_int = ((test_img_sr * 127.5) + 127.5).astype('u1')[0]
cv2.imwrite(test_out_path, test_img_sr_int)

True